In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import math
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn import preprocessing, model_selection

In [6]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/main_merged1.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
merged_df.drop(['Pos', 'Date', 'W_PTS', 'L_PTS', 'DD', 'TD',
       'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'Starter',
       'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'MP', 'FG', 'FGA',
               'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'BLK Last Week', 'BLK Last Month', 'TRB_AVG', 'STL_AVG',
       'FT_perc', 'TRB', 'STL', 'BLK', 'TOV', 'PF', 'W', 'BLK_AVG', 'FG_AVG', 'FGA_AVG', 'FG_perc_AVG', 'FG_AVG Last Week',
       'FGA_AVG Last Week', 'FG_perc_AVG Last Week', 'FG_AVG Last Month', 'AST Last Week', 'AST Last Month', 'AST', 'AST_AVG',
       'FGA_AVG Last Month', 'FG_perc_AVG Last Month'], axis=1, inplace=True)
print(merged_df.columns)
print(merged_df.dtypes)
print(merged_df.shape)
print(merged_df)

Index(['Name', 'Team', 'Home', 'PTS', 'PG', 'SG', 'F', 'C',
       'PTS_AVG Last Month', 'PTS_AVG Last Week', 'PTS_AVG', 'MP_AVG', 'Opp'],
      dtype='object')
Name                   object
Team                   object
Home                    int64
PTS                     int64
PG                      int64
SG                      int64
F                       int64
C                       int64
PTS_AVG Last Month    float64
PTS_AVG Last Week     float64
PTS_AVG               float64
MP_AVG                float64
Opp                    object
dtype: object
(28911, 13)
                  Name Team  Home  PTS  PG  SG  F  C  PTS_AVG Last Month  \
0         Terry Rozier  BOS     1   11   1   0  0  0            0.000000   
1         Kyrie Irving  BOS     1    7   1   0  0  0            0.000000   
2         Marcus Smart  BOS     1    7   1   0  0  0            0.000000   
3       Brad Wanamaker  BOS     1    2   1   0  0  0            0.000000   
4         Jayson Tatum  BOS     1   23   0 

In [7]:
merged_df['Team'] = pd.Categorical(merged_df['Team'])
merged_df['Opp'] = pd.Categorical(merged_df['Opp'])
dummies_team = pd.get_dummies(merged_df['Team'], prefix = 'Team')
dummies_opp = pd.get_dummies(merged_df['Opp'], prefix = 'Opp')
merged_df = pd.concat([merged_df, dummies_team], axis=1)
merged_df = pd.concat([merged_df, dummies_opp], axis=1)

In [5]:
merged_df['Name'] = pd.Categorical(merged_df['Name'])
dummies_name = pd.get_dummies(merged_df['Name'], prefix = '')
merged_df = pd.concat([merged_df, dummies_name], axis=1)
merged_df

,Name,Team,Home,PTS,PG,SG,F,C,PTS_AVG Last Month,PTS_AVG Last Week,...,_Wilson Chandler,_Yante Maten,_Yogi Ferrell,_Yuta Watanabe,_Zach Collins,_Zach LaVine,_Zach Lofton,_Zaza Pachulia,_Zhaire Smith,_Zhou Qi
0,Terry Rozier,BOS,1,11,1,0,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,Kyrie Irving,BOS,1,7,1,0,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,Marcus Smart,BOS,1,7,1,0,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,Brad Wanamaker,BOS,1,2,1,0,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,Jayson Tatum,BOS,1,23,0,0,1,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28906,Royce O'Neale,UTA,0,6,0,0,1,0,5.625000,5.250000,...,0,0,0,0,0,0,0,0,0,0
28907,Tyler Cavanaugh,UTA,0,0,0,0,1,0,0.750000,0.750000,...,0,0,0,0,0,0,0,0,0,0
28908,Ekpe Udoh,UTA,0,14,0,0,0,1,2.437500,4.000000,...,0,0,0,0,0,0,0,0,0,0
28909,Tony Bradley,UTA,0,15,0,0,0,1,0.500000,0.666667,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train = merged_df.loc[1609:25000]
test = merged_df.loc[25000:25500]
X_train = train.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
y_train = train[['PTS']]
X_test = test.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
y_test = test[['PTS']]

In [13]:
bst = xgb.XGBRegressor()
bst.fit(X_train, y_train)
y_pred = bst.predict(X_test)
print(mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))

[19:33:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
37.810191323246194 0.5485607617423953
